In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

d:\workspace\Hackathons\ЦП УрФО 2024\.conda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
embeddings.embed_query("hi")[:5]

[-0.020328126847743988,
 -0.18459415435791016,
 -0.010827395133674145,
 0.05903061851859093,
 0.14664150774478912]

In [49]:
from langchain_core.documents import Document

In [4]:
docs = [Document("a")]

In [50]:
docs = []
with open("reg.txt") as f:
    for line in f.readlines():
        docs.append(Document(line))

In [7]:
# Example
docs = [Document("Машинист поезда № … на приближении к станции …. Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”)."),
        Document("Приказ № … время … (час., мин.). Машинисту поезда № …. Я, дежурный по станции …, разрешаю Вам следовать на … путь при запрещающем показании входного маршрутного) светофора литер …. Маршрут приема готов. ДСП … (фамилия)."),
        Document("Дежурный по станции …. Я, машинист поезда № … (фамилия). Предупредительный … литер перекрылся с зеленого на желтый. Объясните причину.")]

In [51]:
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.vectorstores.utils import DistanceStrategy
db = FAISS.from_documents(docs, embeddings)

In [52]:
from langchain_community.retrievers import BM25Retriever
bm25 = BM25Retriever.from_documents(docs)

In [53]:
query = "Машинист поезда № 2501 на приближении к станции Абзаково"

In [54]:
bm25.invoke(query)

[Document(page_content='Машинист поезда № … на приближении к станции …. Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”).\n'),
 Document(page_content='Машинист поезда № … на подходе к станции …. Вам открыт пригласительный сигнал на входном (маршрутном) светофоре литер … на … свободный путь. Маршрут приема готов. ДСП … (фамилия)\n'),
 Document(page_content='Машинист поезда № …, следующего к станции …. Принимаю Вас на … неспециализированный (или боковой) путь по открытому входному светофору (по пригласительному сигналу или регистрируемому приказу) с остановкой (или без остановки) на станции. ДСП … (фамилия)\n'),
 Document(page_content='Машинист поезда № …, следующего к станции …. По входной стрелке № …, (на … главном пути перегона …, … км … пикет) скорость не более … км/час. ДСП … (фамилия). Только после подтверждения ТЧМ предупре

In [55]:
db.similarity_search_with_score(query, k=5)

[(Document(page_content='Машинист поезда № … на … пути. Маршрутный указатель направления на светофоре … литер неисправен. Маршрут готов в направлении станции …. ДСП … (фамилия)\n'),
  3.4512975),
 (Document(page_content='Машинист поезда № … на … пути станции …. По имеющемуся у Вас письменному разрешению (Путевой записке, разрешениям на бланках ДУ- 54, ДУ-56, ДУ-64) разрешаю отправиться. Маршрут готов. ДСП … (фамилия).\n'),
  3.5646443),
 (Document(page_content='Машинист поезда № … на приближении к станции …. Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”).\n'),
  3.880753),
 (Document(page_content='Готовит маршрут для выезда с пути порядком, установленным ТРА станции или инструкции о порядке обслуживания и организации движения на железнодорожных путях необщего пользования, до первого попутного светофора, далее открывает последу

In [56]:
BM25Retriever.from_documents(db.similarity_search(query, k=5)).invoke(query)

[Document(page_content='Машинист поезда № … на приближении к станции …. Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”).\n'),
 Document(page_content='Машинист поезда № … на … пути. Маршрутный указатель направления на светофоре … литер неисправен. Маршрут готов в направлении станции …. ДСП … (фамилия)\n'),
 Document(page_content='Машинист поезда № … на … пути станции …. По имеющемуся у Вас письменному разрешению (Путевой записке, разрешениям на бланках ДУ- 54, ДУ-56, ДУ-64) разрешаю отправиться. Маршрут готов. ДСП … (фамилия).\n'),
 Document(page_content='Машинист поезда № … на … пути. Повторитель выходного (маршрутного) светофора … литер неисправен, основной светофор открыт. ДСП … (фамилия)\n')]

In [20]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

In [21]:
llm("hi")

"Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [5]:
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

In [17]:
fstring = """Respond Y or N based on how well the following message follows the specified format. Grade only based on the format. Grade Y if you don`t think that this message relate to that group:

Group: {group}
Format: {format}

DATA:
---------
Message: {message}
---------
Write out your explanation for each criterion, then respond with Y or N on a new line."""

In [18]:
criteria1 = {
    "group": "Сообщение",
    "format": "Машинист поезда № … на приближении к станции …. Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”)."
}

In [19]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(fstring)

In [25]:
llm(prompt.format(**criteria1, message="Слушаю Вас, машинист поезда №2501 Федоров"))

"Here's the breakdown of the message against the format:\n\n* Машинист поезда № … на приближении к станции …. : The message starts by addressing the train driver and mentioning their train number (№25). This meets the first criterion.\n* Маршрут на проход по … главному пути готов Входной и выходной (маршрутный и выходной) светофоры открыты. : The message does not mention the route, the main path, or the opening of entrance and exit lights. It only mentions receiving a message from someone. This does not meet the second criterion.\n* ДСП … (фамилия) (сообщения передаются только машинистам пассажирских поездов, работающим в “одно лицо”). : The message does not mention the dispatcher's name or specify that it is only for train drivers of passenger trains working solo. This also does not meet the third criterion.\n\nResponse: N"

In [ ]:
def check(message):
    

In [12]:

evaluator1 = load_evaluator(EvaluatorType.CRITERIA, llm=llm, criteria=criteria1, prompt=prompt)